# Building the GPT Tokenizer

## 0. Some Pre-requisites

### Introduction to Tokenization
Tokenization is the process of breaking down a piece of text into smaller units called **tokens**. These tokens are the vocabulary of the language model. 

#### Why is Tokenization so Important?

- **Spelling, String Processing, and Arithmetic:** LLMs often struggle with tasks that seem simple to humans like spelling words correctly reversing a string, or performing basic math. This is because these tasks require operating on individual characters but the model sees the world in terms of "tokens," which are often chunks of words. For example the number 1275 might be a single token not four separate digit characters making it hard for the model to perform arithmetic on it.

- **Non-English Languages and Code:** The vocabulary of a tokenizer is typically trained on a massive corpus of text which is often predominantly English. This means other languages or programming languages might be broken down into less efficient or meaningful tokens making the model less effective at understanding and generating them.

- **Weird Errors and Warnings:** Seemingly random failures or warnings like those related to "trailing whitespace" or specific strings like "SolidGoldMagikarp", are often artifacts of how the tokenizer processes text before the model ever sees it.

- **The Dream of a Tokenizer-Free World:** A major goal in AI research is to create models that can operate directly on raw text (or even raw bytes) which would eliminate the entire layer of complexity and the associated problems introduced by tokenization.



### Unicode and UTF-8 Encoding

- **Unicode:** This is a universal standard that assigns a unique number, called a code point to almost every character symbol or emoji in every language. 
- **UTF-8:** This is an encoding scheme that specifies how to represent these Unicode code points as a sequence of bytes (integers from 0 to 255). Simple characters like those in the English alphabet can be represented by a single byte while more complex characters or emojis might require multiple bytes.

## 1. Basic Text Encoding

### Unicode Code Points (ord)
The `ord()` function in Python gives us the Unicode code point for a given character. 

In [1]:
"안녕하세요 👋 (hello in Korean!)"

'안녕하세요 👋 (hello in Korean!)'

In [2]:
[ord(x)for x in "안녕하세요 👋 (hello in Korean!)"]

[50504,
 45397,
 54616,
 49464,
 50836,
 32,
 128075,
 32,
 40,
 104,
 101,
 108,
 108,
 111,
 32,
 105,
 110,
 32,
 75,
 111,
 114,
 101,
 97,
 110,
 33,
 41]

### UTF-8 Encoded Bytes
We take the same string and encode it into a sequence of bytes. 
- Notice that the English characters and symbols (like `h`, `e`, `l`, `o`,`     `) are represented by single bytes (numbers less than 128) that match their ASCII values.
- The Korean characters and the emoji are represented by sequences of multiple bytes (numbers greater than 127). For example the waving hand emoji `👋` becomes the four-byte sequence [240, 159, 145, 139].

In [3]:
list("안녕하세요 👋 (hello in Korean!)".encode("utf-8"))

[236,
 149,
 136,
 235,
 133,
 149,
 237,
 149,
 152,
 236,
 132,
 184,
 236,
 154,
 148,
 32,
 240,
 159,
 145,
 139,
 32,
 40,
 104,
 101,
 108,
 108,
 111,
 32,
 105,
 110,
 32,
 75,
 111,
 114,
 101,
 97,
 110,
 33,
 41]

## 2. Implementing Byte-Pair Encoding

The core idea of BPE is to start with a simple vocabulary (all individual bytes) and iteratively merge the most frequent pair of adjacent tokens to create new and longer tokens. This allows the model to learn a vocabulary that is optimized for the specific text it is trained on creating a balance between character-level and word-level tokenization.

### Initial Text and Byte Conversion
We start with a sample text and convert it into its raw UTF-8 byte representation. This list of integers (from 0 to 255) is our initial sequence of tokens. Our goal is to compress this token sequence compress by merging frequent pairs.

In [4]:
text = "Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception."
tokens = text.encode("utf-8")# raw bytes
# converting to a list of integers in range 0..255 for convenience
tokens = list(map(int, tokens))
print('---')
print(text)
print("length:", len(text))# number of characters
print('---')
print(tokens)# list of integers in range 0..255
print("length:", len(tokens))

---
Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software (whatever that means—like using wchar_t for all the strings, right?). But Unicode can be abstruse, and diving into the thousand-page Unicode Standard plus its dozens of supplementary annexes, reports, and notes can be more than a little intimidating. I don’t blame programmers for still finding the whole thing mysterious, even 30 years after Unicode’s inception.
length: 533
---
[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159, 135, 180, 226, 128, 140

### Finding the Most Frequent Pair(`get_stats`)
Here we do the first step in a BPE merge iteration. It takes our current list of token IDs and counts the occurrences of every adjacent pair. The result is a dictionary where each key is a pair (`token1`, `token2`) and the value is its frequency.

In [5]:
def get_stats(ids):
    counts = {}# dictionary mapping (token1, token2) -> count
    for pair in zip(ids, ids[1:]): # for each adjacent pair
        counts[pair] = counts.get(pair, 0) + 1 # increment its count
    return counts

stats = get_stats(tokens)
print(stats)

{(239, 188): 1, (188, 181): 1, (181, 239): 1, (239, 189): 6, (189, 142): 1, (142, 239): 1, (189, 137): 1, (137, 239): 1, (189, 131): 1, (131, 239): 1, (189, 143): 1, (143, 239): 1, (189, 132): 1, (132, 239): 1, (189, 133): 1, (133, 33): 1, (33, 32): 2, (32, 240): 3, (240, 159): 15, (159, 133): 7, (133, 164): 1, (164, 240): 1, (133, 157): 1, (157, 240): 1, (133, 152): 1, (152, 240): 1, (133, 146): 1, (146, 240): 1, (133, 158): 1, (158, 240): 1, (133, 147): 1, (147, 240): 1, (133, 148): 1, (148, 226): 1, (226, 128): 12, (128, 189): 1, (189, 32): 1, (159, 135): 7, (135, 186): 1, (186, 226): 1, (128, 140): 6, (140, 240): 6, (135, 179): 1, (179, 226): 1, (135, 174): 1, (174, 226): 1, (135, 168): 1, (168, 226): 1, (135, 180): 1, (180, 226): 1, (135, 169): 1, (169, 226): 1, (135, 170): 1, (170, 33): 1, (159, 152): 1, (152, 132): 1, (132, 32): 1, (32, 84): 1, (84, 104): 1, (104, 101): 6, (101, 32): 20, (32, 118): 1, (118, 101): 3, (101, 114): 6, (114, 121): 2, (121, 32): 2, (32, 110): 2, (110,

In [6]:
print(sorted(((v, k) for k, v in stats.items()), reverse=True)[:10])# top 10 most frequent pairs

[(20, (101, 32)), (15, (240, 159)), (12, (226, 128)), (12, (105, 110)), (10, (115, 32)), (10, (97, 110)), (10, (32, 97)), (9, (32, 116)), (8, (116, 104)), (7, (159, 135))]


In [7]:
top_pair = max(stats, key=stats.get)# the most frequent pair
print(top_pair)

(101, 32)


### Merging the Top Pair
The `merge` function performs the second step of the BPE iteration. It takes the list of token IDs, the most frequent pair and a new token ID (`idx`) and it replaces every occurrence of the pair with the new ID.
- **The logic:** It iterates through the list. If it finds the target pair it appends the new `idx` to the output list and skips forward two positions. Otherwise it just appends the current token.
- **The result:** A new and shorter list of tokens where the most common pair has been compressed into a single new token. In this example we replace all (101, 32) pairs with the new token 256.

In [8]:
def merge(ids, pair, idx):
  # in the list of ints (ids), replace all consecutive occurences of pair with the new token idx
  newids = []
  i = 0
  while i < len(ids):
    # if we are not at the very last position AND the pair matches, replace it
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])# copy over the current token
      i += 1# move forward by one
  return newids
print(merge([5, 6, 6, 7, 9, 1], (6, 7), 99))


[5, 6, 99, 9, 1]


In [9]:
tokens2 = merge(tokens, top_pair, 256)
print(tokens2)

[239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159, 135, 180, 226, 128, 140, 240, 159, 135, 169, 226, 128, 140, 240, 159, 135, 170, 33, 32, 240, 159, 152, 132, 32, 84, 104, 256, 118, 101, 114, 121, 32, 110, 97, 109, 256, 115, 116, 114, 105, 107, 101, 115, 32, 102, 101, 97, 114, 32, 97, 110, 100, 32, 97, 119, 256, 105, 110, 116, 111, 32, 116, 104, 256, 104, 101, 97, 114, 116, 115, 32, 111, 102, 32, 112, 114, 111, 103, 114, 97, 109, 109, 101, 114, 115, 32, 119, 111, 114, 108, 100, 119, 105, 100, 101, 46, 32, 87, 256, 97, 108, 108, 32, 107, 110, 111, 119, 32, 119, 256, 111, 117, 103, 104, 116, 32, 116, 111, 32, 226, 128, 156

In [10]:
print("length:", len(tokens2))

length: 596
